In [4]:

import tensorflow as tf
import numpy as np
from os import listdir
from os.path import isfile, join
import pandas as pd
import os
import sys

from functions.audio_functions import *
from tf_classes.RNN_class import Multilayer_RNN as Threelayer_RNN

In [ ]:
words = ['dog','four','left','off','seven','three','wow','bed','down','go','marvin','on','sheila','tree','yes','bird','eight','happy','nine','one','six','two','zero','cat','five','house','no','right','stop','up','silence']
# reduced label indices
results = ['unknown','left', 'off' ,'down', 'go', 'on', 'yes', 'no' ,'right' ,'stop' ,'up' ,'silence']


In [ ]:
labels_to_numbers = {}
for f in range(len(results)):
    labels_to_numbers[results[f]] = f

    
location = '../data/train/audio/'

index_dict = {}
index = 0
alt_label = 'unknown'


# load data from directories

for label in words:
    if label in redux:
        for f in listdir('%s%s/' %(location,label)):
        #print(label, " ",f)
            index_dict[index] = (label,'%s%s/%s'%(location,label,f))
            index +=1
    else:
        print(label)
        for f in listdir('%s%s/' %(location,label)):
            index_dict[index] = (alt_label,'%s%s/%s'%(location,label,f))
            index +=1

index_df = pd.DataFrame.from_dict(index_dict, orient='index')

In [ ]:
index_df.head()

In [ ]:

print(index_df[0].unique())

In [ ]:

# load sample data

sample_sub = pd.read_csv('../sample_submission.csv')

sample_sub[0] = sample_sub['label'].apply(lambda x: 'unknown')
sample_sub[1] = sample_sub['fname'].apply(lambda x: '../data/test/audio/%s' %(x))

sample_sub = sample_sub.drop('fname',axis=1)
sample_sub = sample_sub.drop('label',axis=1)


index_sample = []
for i in range(len(sample_sub)):
    index_sample.append(i)

In [ ]:

real_idx = []
for i in range(len(realDat)):
    real_idx.append(i)

prefetched_real = preload_data(real_idx,realDat,augment=False)
X_mess, y_mess = batch_from_prefetch_samples(prefetched_real,len(real_idx))



In [ ]:

train_idx, val_idx = split_val_data(0.99) # split the training data into test and validation



In [ ]:
# takes a lot of time to load data from mfcc,  prefetch to numpy

% time prefetched_data = preload_data(train_idx, index_df ,augment=False)

In [ ]:
% time X_batch, y_batch = batch_from_prefetch_samples(prefetched_data,90)

In [ ]:
% time prefetched_val = preload_data(val_idx, index_df ,augment=False)

In [ ]:
% time prefetched_test = preload_data(index_sample,sample_sub, augment=False)

In [ ]:
%time X_batch, y_batch = batch_from_prefetch_samples(prefetched_data,100)

In [ ]:
%time data,y_empty = fulldata_prefetch_samples_inorder(prefetched_test)

In [ ]:
X_batch, y_batch = batch_from_prefetch_samples(prefetched_data,100)
X_val,y_val = batch_from_prefetch_samples(prefetched_val,len(val_idx)

In [7]:
RNN = Threelayer_RNN(an_inputs = 99*13,an_hidden1 = 800,
                 an_hidden2 = 550, an_hiddencore = 200,a_learning_rate_initial = 0.00007,
                 a_decay_steps = 10000,a_decay_rate = 0.95,
                 l2_reg = 0.0004,conv_learning_rate_init = 0.002,conv_decay_steps = 10000,
                 conv_decay_rate = 0.95,conv_learning_rate = 0.002,dropout = 0.5,
                 input_keep_prob = 0.3,n_steps = 13,n_inputs = 99,
                 n_neurons = 300 ,n_outputs = 7,
                 initial_learning = 0.00015,decay_steps = 30000,
                 decay_rate = 0.98,momentum = 0.995 ,scale =0.0014,
                logdir = 'Tensorboard_logdir')
RNN._build()

In [ ]:
RNN.run_autoencoder(prefetched_test=prefetched_data,X_mess=X_val)
RNN.run_convnet(X_val=X_val,y_val=y_val,prefetched_data=prefetched_data)
RNN.run(X_val=X_val,y_val=y_val,prefetched_data=prefetched_data)


In [ ]:
RNN.predict(X_test,y_test)